In [8]:
import torch

In [9]:
import pandas as pd

# df = pd.read_parquet("hf://datasets/pankajemplay/llama-intent-1615/data/train-00000-of-00001-b2b83bc2e0f680a6.parquet")
df = pd.read_csv("intent.csv")


In [10]:
df.head()

,User Query,Intent,Bot Response
0,Hello,welcome,Hello! How can I assist you today?
1,I want to cancel my order,order detail,Please provide the order number or deal ID to ...
2,I'm not satisfied with this.,raise question,I'm sorry to hear that. How can I assist you f...
3,Where can I find the user manual?,search,Searching for the user manual...
4,How's the weather today?,small talk,I'm not sure about the current weather.


In [11]:
df = df[["User Query","Intent","Bot Response"]]

In [12]:
df.head()

,User Query,Intent,Bot Response
0,Hello,welcome,Hello! How can I assist you today?
1,I want to cancel my order,order detail,Please provide the order number or deal ID to ...
2,I'm not satisfied with this.,raise question,I'm sorry to hear that. How can I assist you f...
3,Where can I find the user manual?,search,Searching for the user manual...
4,How's the weather today?,small talk,I'm not sure about the current weather.


In [13]:
# df.to_csv('intent.csv',index=False)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User Query    1615 non-null   object
 1   Intent        1615 non-null   object
 2   Bot Response  1615 non-null   object
dtypes: object(3)
memory usage: 38.0+ KB


In [15]:
import pandas as pd
import json
from collections import defaultdict

# Read the CSV file into a DataFrame
# df = pd.read_csv('your_file.csv')

# Initialize a dictionary to store intents
intents_dict = defaultdict(lambda: {"patterns": [], "responses": []})

# Iterate over the DataFrame rows to fill the dictionary
for _, row in df.iterrows():
    intent = row['Intent']
    user_query = row['User Query']
    bot_response = row['Bot Response']
    
    # Append the user query (pattern) and response to the corresponding intent
    intents_dict[intent]["patterns"].append(user_query)
    intents_dict[intent]["responses"].append(bot_response)

# Convert the intents dictionary into the desired JSON format
intents_json = {"intents": [{"intent": intent, "patterns": data["patterns"], "responses": data["responses"]} for intent, data in intents_dict.items()]}

# Write the JSON data to a file
with open('intents.json', 'w') as json_file:
    json.dump(intents_json, json_file, indent=4)

print("Conversion to intents.json completed!")


Conversion to intents.json completed!


In [16]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

# 1. Load the CSV data
# Assuming you have a CSV file with columns: 'User Query', 'Intent', 'Bot Response'
# file_path = 'your_file.csv'  # Replace with the actual file path
# df = pd.read_csv(file_path)

# Inspect the data
print(df.info())  # Shows dataframe structure and memory usage

# 2. Tokenize and Vectorize using Bag-of-Words (BoW)
vectorizer = CountVectorizer()  # You can also use TF-IDF by replacing this with TfidfVectorizer
X = vectorizer.fit_transform(df['User Query']).toarray()  # Tokenize and convert to BoW vectors

# 3. Label Encoding for Intent column
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Intent'])  # Convert intents to numerical labels

# 4. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Create a Custom Dataset class for PyTorch
class ChatbotDataset(Dataset):
    def __init__(self, queries, intents):
        self.queries = queries
        self.intents = intents

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        return self.queries[idx], self.intents[idx]

# Create train and test datasets
train_dataset = ChatbotDataset(X_train_tensor, y_train_tensor)
test_dataset = ChatbotDataset(X_test_tensor, y_test_tensor)

# 6. Create DataLoaders for batching
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Sample output to verify
for queries, intents in train_loader:
    print("Batch of Queries (BoW):", queries)
    print("Batch of Intents:", intents)
    break


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User Query    1615 non-null   object
 1   Intent        1615 non-null   object
 2   Bot Response  1615 non-null   object
dtypes: object(3)
memory usage: 38.0+ KB
None
Batch of Queries (BoW): tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Batch of Intents: tensor([7, 2, 9, 2, 0, 2, 4, 8, 7, 6, 9, 3, 4, 9, 1, 9])


In [24]:
import torch.nn as nn
import torch.optim as optim

# 1. Define the Model
class ChatbotModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ChatbotModel, self).__init__()
        self.fc = nn.Linear(input_size, 128)  # Hidden layer with 128 units
        self.relu = nn.ReLU()
        self.output = nn.Linear(128, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        x = self.output(x)
        return x

# Set parameters
input_size = X_train.shape[1]  # Number of features (vocabulary size)
num_classes = len(label_encoder.classes_)  # Number of unique intents
print(input_size,num_classes)

# Initialize the model
model = ChatbotModel(input_size, num_classes)

# 2. Set Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# 3. Training Loop
num_epochs = 10  # Number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for queries, intents in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(queries)  # Forward pass
        loss = criterion(outputs, intents)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()  # Accumulate loss
    if epoch %10 ==0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# 4. Evaluation
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for evaluation
    for queries, intents in test_loader:
        outputs = model(queries)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += intents.size(0)  # Total samples
        correct += (predicted == intents).sum().item()  # Count correct predictions

accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

torch.save(model.state_dict(), 'chatbot_model.pth')
print("Model saved as chatbot_model.pth")


1539 11
Epoch [1/10], Loss: 2.0753
Accuracy on test set: 81.11%
Model saved as chatbot_model.pth


In [23]:
# 1. Prepare the test input
def preprocess_input(user_input):
    # Transform user input into the same format as training data
    input_vector = vectorizer.transform([user_input]).toarray()
    return torch.tensor(input_vector, dtype=torch.float32)

# 2. Make Predictions
def predict_intent(user_input):
    model.eval()  # Set the model to evaluation mode\
    with torch.no_grad():
        input_tensor = preprocess_input(user_input)
        output = model(input_tensor)
        _, predicted = torch.max(output.data, 1)
        return label_encoder.inverse_transform(predicted.numpy())[0]

# 3. Retrieve Responses
def get_bot_response(intent):
    response = df[df['Intent'] == intent]['Bot Response'].values[0]
    return response

# 4. Interactive Testing


In [19]:
print("Chatbot is ready to chat! Type 'exit' to end.")
while True:
    user_input = input("You: ")
    print(f"You : {user_input}")
    if user_input.lower() == 'exit':
        break
    predicted_intent = predict_intent(user_input)
    bot_response = get_bot_response(predicted_intent)
    print("Bot:", bot_response)

Chatbot is ready to chat! Type 'exit' to end.
You : exit


In [27]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load your trained model
input_size = 1539  # This should be set to the size of your input features
num_classes = 11  # This should be set to the number of classes in your label encoder

model = ChatbotModel(input_size, num_classes)
model.load_state_dict(torch.load('chatbot_model.pth'))
model.eval()  # Set model to eval mode

# Load your CSV with Intent and Bot Response
df = pd.read_csv('intent.csv')  # Replace with actual CSV path

# Simulating vectorizer and label encoder (you should load them if you have them saved)
vectorizer = CountVectorizer()
label_encoder = LabelEncoder()

# Define FastAPI app
app = FastAPI()

# Pydantic model for incoming user messages
class Query(BaseModel):
    message: str

# Preprocess the user input (tokenization)
def preprocess_input(user_input):
    input_vector = vectorizer.transform([user_input]).toarray()
    return torch.tensor(input_vector, dtype=torch.float32)

# Predict intent
def predict_intent(user_input):
    with torch.no_grad():
        input_tensor = preprocess_input(user_input)
        output = model(input_tensor)
        _, predicted = torch.max(output.data, 1)
        intent = label_encoder.inverse_transform(predicted.numpy())[0]
        return intent

# Get bot response for the predicted intent
def get_bot_response(intent):
    response = df[df['Intent'] == intent]['Bot Response'].values[0]
    return response

# Define the /chat/ endpoint for user messages
@app.post("/chat/")
async def chat(query: Query):
    user_message = query.message
    predicted_intent = predict_intent(user_message)
    bot_response = get_bot_response(predicted_intent)
    return {"intent": predicted_intent, "response": bot_response}


/tmp/ipykernel_4891/4275302497.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('chatbot_model.pth'))


In [10]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Define the model architecture again (must match the one used during training)
class ChatbotModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ChatbotModel, self).__init__()
        self.fc = nn.Linear(input_size, 128)  # Hidden layer with 128 units
        self.relu = nn.ReLU()
        self.output = nn.Linear(128, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        x = self.output(x)
        return x

# Load your trained model
input_size = 1539  # This should be set to the size of your input features
num_classes = 11# This should be set to the number of classes in your label encoder

model = ChatbotModel(input_size, num_classes)
model.load_state_dict(torch.load('chatbot_model.pth'))
model.eval()  # Set model to eval mode

# Load your CSV with Intent and Bot Response
df = pd.read_csv('intent.csv')  # Replace with actual CSV path

# Simulating vectorizer and label encoder (you should load them if you have them saved)
# vectorizer = CountVectorizer()
# label_encoder = LabelEncoder()

vectorizer = CountVectorizer()  # You can also use TF-IDF by replacing this with TfidfVectorizer
X = vectorizer.fit_transform(df['User Query']).toarray()  # Tokenize and convert to BoW vectors

# 3. Label Encoding for Intent column
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Intent'])


# Preprocess the user input (tokenization)
def preprocess_input(user_input):
    input_vector = vectorizer.transform([user_input]).toarray()
    return torch.tensor(input_vector, dtype=torch.float32)

# Predict intent
def predict_intent(user_input):
    with torch.no_grad():
        input_tensor = preprocess_input(user_input)
        output = model(input_tensor)
        _, predicted = torch.max(output.data, 1)
        intent = label_encoder.inverse_transform(predicted.numpy())[0]
        return intent

# Get bot response for the predicted intent
def get_bot_response(intent):
    response = df[df['Intent'] == intent]['Bot Response'].values[0]
    return response



print("Chatbot is ready to chat! Type 'exit' to end.")
while True:
    user_input = input("You: ")
    print(f"You : {user_input}")
    if user_input.lower() == 'exit':
        break
    user_input=repr(user_input)
    predicted_intent = predict_intent(user_input)
    bot_response = get_bot_response(predicted_intent)
    print("Bot:", bot_response)


/tmp/ipykernel_10517/1249130460.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('chatbot_model.pth'))


Chatbot is ready to chat! Type 'exit' to end.
You : hello
Bot: Hello! How can I assist you today?
You : hi
Bot: Hello! How can I assist you today?
You : hi
Bot: Hello! How can I assist you today?
You : order number 2
Bot: Please provide the order number or deal ID to cancel.
You : 
Bot: I'm not sure about the current weather.
You : 1234
Bot: I'm not sure about the current weather.
You : 
Bot: I'm not sure about the current weather.
You : exit
